In [33]:
import numpy as np
import mne
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
import pyeeg as p
from numpy import nan
import math

In [34]:
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

In [35]:
def mean_kurtosis(df):
    kurtosis_vals = kurtosis(df)
    return np.mean(kurtosis_vals)

In [36]:
def mean_skewness(df):
    skew_vals = skew(df)
    return np.mean(skew_vals)

In [37]:
def mean_pfd(df):
    pfd_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        pfd_val = p.pfd(col)
        pfd_vals.append(pfd_val)
    return np.mean(pfd_vals)

In [38]:
def mean_hjorth_mob_comp(df):
    mob_vals = []
    comp_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        mob_col, comp_col = p.hjorth(col)
        mob_vals.append(mob_col)
        comp_vals.append(comp_col)
    return np.mean(mob_vals), np.mean(comp_vals)

In [39]:
def all_psd(data):
    fs = 256                                
    N = data.shape[1] # total num of points 

    # Get only in postive frequencies
    fft_vals = np.absolute(np.fft.rfft(data))

    n_rows = fft_vals.shape[0]
    n_cols = fft_vals.shape[1]
    psd_vals = np.zeros(shape=(n_rows, n_cols))

    for i in range(n_rows):
        for j in range(n_cols):
            psd_vals[i][j] = (N/fs) * fft_vals[i][j] * fft_vals[i][j];


    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(data.shape[1], 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    psd_vals_list = []
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(psd_vals[:,freq_ix])
        psd_vals_list.append(eeg_band_fft[band] * 1000000)
    return psd_vals_list

In [40]:
def sum_psd(data):
    psd_vals = all_psd(data)
    return np.sum(psd_vals)

In [41]:
def mean_spectral_entropy(data):
    psd_vals = all_psd(data)
    power_ratio = []
    sum_psd_vals = sum_psd(data)
    for val in psd_vals:
        power_ratio.append(val/sum_psd_vals)
    bands = [0,4,8,12,30,45]
    Fs = 256
    spec_entropy = p.spectral_entropy(data, bands, Fs, power_ratio)
    return spec_entropy

In [46]:
def add_row(df, data, start, end):
    row_to_add = []
    
    d = df.loc[(df['time'] >= start) & (df['time'] < end)]
    data = data[:,256*start:256*end]
    data = data[:][0]
    
    mean_var = mean_variance(d)
    mean_k = mean_kurtosis(d)
    mean_skew = mean_skewness(d)
    pfd = mean_pfd(d)
    h_mob, h_comp = mean_hjorth_mob_comp(df)
    mean_spec = mean_spectral_entropy(data)
    
    row_to_add.append(mean_var)
    row_to_add.append(mean_k)
    row_to_add.append(mean_skew)
    row_to_add.append(pfd)
    row_to_add.append(h_mob)
    row_to_add.append(h_comp)
    row_to_add.append(mean_spec)
    row_to_add.append(1)
    
    
    return row_to_add

In [47]:
data = mne.io.read_raw_edf('chb01_04.edf', preload=True)
header = ','.join(data.ch_names)
df = data.to_data_frame()

Extracting EDF parameters from C:\Users\upama\BTP\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<ipython-input-47-57fbe5df9d8d>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_04.edf', preload=True)


In [57]:
#processes one seizure in 10s windows
#adds rows with features extracted from these windows
start = temp = 1467 
duration = 10
end = 1494

df = df.loc[(df['time'] >= start) & (df['time'] < end)]

res = pd.DataFrame()
with open('test.csv', 'w') as file:
    writer = csv.writer(file)
    while temp < end:
        row = add_row(df, data, temp, temp + duration)
        res=res.append(pd.Series(row),ignore_index=True)
        writer.writerow(row)
        temp += duration

res.columns = ['Variance', 'Kurtosis', 'Skewness', 'Petrosian Fractal Dimension', 'Hjorth Mobility', 'Hjorth Complexity', 'Spectral Entropy', 'Label']
res

,Variance,Kurtosis,Skewness,Petrosian Fractal Dimension,Hjorth Mobility,Hjorth Complexity,Spectral Entropy,Label
0,10.274191,-2.000000,-3.660378e-15,1.000000,0.196882,5.750124,0.289082,1.0
1,57.120292,-1.500000,-5.490877e-02,0.892538,0.196882,5.750124,0.330675,1.0
2,14.208429,-2.041667,2.306836e-16,1.000000,0.196882,5.750124,0.184555,1.0
